In [1]:
import torch
from hydra import initialize, compose
from hydra.utils import instantiate
from src.data import DataHandler
from src.model import ResNet, ConvNext, Dummy, ModelTrainer

In [2]:
import os
os.chdir('/Users/maciej.filanowicz/CellPainting/src')

In [3]:
initialize(version_base="1.2", config_path="src/conf", job_name="test_app")
cfg = compose(config_name="config", return_hydra_config=True)

In [4]:
dataset_config = instantiate(cfg.dataset)

In [5]:
train_config = instantiate(cfg.train)

In [6]:
data_loaders = DataHandler(dataset_config).get_data_loaders()

In [7]:
# model = torch.hub.load("pytorch/vision", "resnet18")

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")

In [9]:
model = Dummy()
trainer = ModelTrainer(model,train_config,**data_loaders)

In [10]:
trainer.run_trainer()

Progress:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/162 [00:00<?, ?it/s]